In [1]:
import numpy as np
import pandas as pd
import pickle
from matplotlib import pyplot as plt
from esem import rf_model, gp_model
from scipy import stats

2022-02-24 11:19:27.338097: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-02-24 11:19:27.338132: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


# Load dataset

In [2]:
train = np.loadtxt("train.txt")
Y = train[:, 0]
X = train[:, 1:] # 1: for all, 1:7 for no date features.
print(Y.shape)
print(X.shape)

(1005906,)
(1005906, 6)


In [3]:
# Define training and test inds
n_train = int(Y.shape[0] * 0.8) # 80 % training
training_inds = np.arange(0, n_train)
testing_inds = np.arange(n_train, len(Y))
n_test = len(testing_inds)

print(n_train)
print(n_test)

804724
201182


# Define training data set

In [4]:
X_train = X[training_inds, :]
Y_train = Y[training_inds]

# Train model

In [ ]:
%%time
# RF model
rf_model = rf_model(X_train, Y_train) # Default
# rf_model = rf_model(X_train, Y_train, n_estimators=200) # No significant improvement over default.
rf_model.train()

# GP model (can't get it to work)
# rf_model = gp_model(X_train, Y_train, kernel=['White'])
# rf_model.train()

2022-02-24 11:19:44.020362: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-02-24 11:19:44.020401: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-02-24 11:19:44.020424: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (shas0319.rc.int.colorado.edu): /proc/driver/nvidia/version does not exist
2022-02-24 11:19:44.021108: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
/projects/erke2265/software/anaconda/envs/esem/lib/python3.7/site-packages/esem/model_adaptor.

# Save model

In [ ]:
# # The sklearn model is held internally in the esem model
# with open("rf_model.pkl","wb") as f:
#     pickle.dump(rf_model.model.model, f)

# Define testing data set

In [ ]:
X_test = X[testing_inds, :]

# Inference

In [ ]:
%%time
Y_rf_test = rf_model.predict(X_test)[0]

# Perform analysis

In [ ]:
x_1_to_1 = [-9999, 9999]
y_1_to_1 = [-9999, 9999]

In [ ]:
def calc_rmse(predictions, targets):
    return np.sqrt(((predictions - targets) ** 2).mean())

def calc_bias(predictions, targets):
    return (predictions - targets).mean()

In [ ]:
# Random Forest 
x = Y[testing_inds] * 100 # Truth (cm)
y = Y_rf_test * 100 # Prediction (cm)
lin_reg = stats.linregress(x, y)
rmse = calc_rmse(y, x)
bias = calc_bias(y, x)

# Limit
lim = np.max([np.abs(x).max(), np.abs(y).max()])

plt.figure(figsize=(9, 7))
plt.hexbin(x, y, extent=(-lim, lim, -lim, lim), cmap=plt.cm.Blues, bins='log', gridsize=(40, 40))
plt.xlim([-lim, lim])
plt.ylim([-lim, lim])
cb = plt.colorbar()
cb.set_label("Count", fontsize=22)
cb.ax.tick_params(labelsize=20) 
plt.plot(x, lin_reg.intercept + lin_reg.slope*x, 'r', label='fitted line', linewidth=3)
plt.plot(x_1_to_1, y_1_to_1, 'k--', linewidth=2)
plt.grid()
plt.xlabel("SNOWPACK: 91 day FAC change [cm]", fontsize=20)
plt.ylabel("ML: 91 day FAC change [cm]", fontsize=20)
plt.xticks(fontsize=16)
plt.yticks(fontsize=16)
plt.title("Slope = " + str(lin_reg.slope) + \
          "\nR^2 = " + str(lin_reg.rvalue**2) + \
          ",\np = " + str(lin_reg.pvalue) + \
          ",\nRMSE = " + str(rmse) + \
          ",\nBias = " + str(bias) + \
          ",\nN = " + str(n_test) + \
          ",\nX-axis Mean Magnitude = " + str(np.abs(x).mean()),
         fontsize=20)

In [ ]:
diff = y - x
limit = np.abs(diff).max()
print(limit)

# Histogram
plt.figure(figsize=(7, 5))
plt.hist(diff, bins=75)
plt.grid()
plt.xlim([-limit, limit])
plt.xlabel("ML predicted FAC change error [cm]", fontsize=20)
plt.ylabel("Count", fontsize=20)
plt.xticks(fontsize=16)
plt.yticks(fontsize=16)